# AiDM 2019: Assignment 1 - Recommender Systems

## Part 2: Matrix factorization

Auke Bruinsma, s1594443 and Simon van Wageningen, s2317079

As instructed, we have divided the first assignment of AiDM into two parts: The first on the naive models and this part on the matrix factorization. The structure of this part will be as follows: Before each cell of code we'll provide background info in markdown cells if necessary. If a cell produces relevant output we'll write this down. At the end there will be a short conclusion/discussion section where we compare our results of the naive models and matrix factorization with each other.

In [5]:
# Imports
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from numpy import genfromtxt
import sys

In the following cell we import the three datasets using pandas. We have given each columns the names that are found in the readme file so that each column can be accessed with, for example, ratings['UserID'].

In [6]:
# Import datasets and infer a header with the correct column labels.
ratings = pd.read_csv(filepath_or_buffer='data/ml-1m/ratings.dat',sep='::',header=None,names=['UserID','MovieID','Rating','Timestamp'],engine='python')
users = pd.read_csv(filepath_or_buffer='data/ml-1m/users.dat',sep='::',header=None,names=['UserID','Gender','Age','Occupation','Zip-code'],engine='python')
movies = pd.read_csv(filepath_or_buffer='data/ml-1m/movies.dat',sep='::',header=None,names=['MovieID','Title','Genres'],engine='python')

### 1. Create the array $X$

Now that we have imported all necessary packages and datasets, we can start with the assignment. After a quick look at the data, we noticed the following things:

- There are approximately 6000 users.
- There are approximately 4000 movies.
- Not every users has rated each movie.
- Sometimes the ['MovieID'] column skips a number. This may mean that a movie is deleted from the database, or that no one has rated that movie. 

In order to complete the matrix factorization, we will create a matrix called $X$. This matrix has approximately the size $(6000,4000)$, where $X_{i,j}$ denotes how the $i_{th}$ user will rate the $j_{th}$ movie. Because of the last two points in the previous enumeration, there are a lot of empty elements in this matrix. Approximately 4.2 percent of the matrix $X$ has elements that are not equal to zero, which means 4.2 percent has been rated.

In [7]:
# Create array X with at X[i][j] the ratings for how the ith user would rate the jth movies
X = np.zeros((users['UserID'][len(users)-1]+1,movies['MovieID'][len(movies)-1]+1))

# Fill array.
for k in range(len(ratings)):
    X[ratings['UserID'][k]][ratings['MovieID'][k]] = ratings['Rating'][k]

In this cell we compute the percentage filled of the matrix $X$.

In [8]:
# To get a better overview, here we print the number of matrix elements that is known.
print('Percentage filled: {0} %'.format(100*len(ratings)/(len(X)*len(X[0]))))

Percentage filled: 4.188467095557036 %


The exact shape of the matrix $X$ is (6041,3953), it is also printed for a quick overview. You can see that that most elements are zero and only two elements have a rating.

In [9]:
print('Shape X: {0}\n'.format(X.shape))
print(X)

Shape X: (6041, 3953)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 3. 0. ... 0. 0. 0.]]


Now we have the imported data in a matrix called $X$. This matrix will form the basis of the computations that are needed to perform the matrix factorization. However, first we need to set up the N-fold cross validation.

### 2. N-fold cross-validation:

1. Split your data in $N$ parts (equal size);
2. Develop $N$ models on all combinations of $(N-1)$ parts;
3. Test each model on the remaining parts (test sets);
4. Average the errors over these $N$ test sets;
5. The average error is a realistic estimator of the error made by your model on the fresh data.

#### In practice:
- $N=5$ (5-fold cross-validation) or $(N=10)$ (10-fold cross-validation)
- Errors also measured on the training sets/folds
- Standard deviation of errors says something about 'model stability'

The text in the previous markdown cell is taken from the slides. The idea of N-fold cross validation is to create your own training and testing set so that you can test the accuracy of your model. You divide all your data into $N$ parts, in our case 5 parts. Then 4/5 is used as a training set and 1/5 as a testing set. You repeat this process for each one-fifth of the data, so that each one-fifth is used for testing, and each 4/5 is used for the training set. This will give an accurate outcome of the model.

The splitting of the data in 5 parts is done in the following way: All the indices of the elements of the matrix $X$ which are not equal to zero, are stored in a list called non_zero_indices. Each element of this array contains the $i$-coordinate, $j$-coordinate, and also the rating. This means the size of this array is $(1000209,3)$, since there are $1000209$ ratings present in the matrix $X$. Next, the non_zero_indices array is randomly shuffled, to make the results more valid. The array is then split into 5 parts, and this array is returned by the function k_fold_cross_validation. The size of this list is $(n=5,~10^6/(n=5),3)$.

In [10]:
# Create an array where all the indices will be stored which are not zero.
non_zero_indices = []
for i in range(len(X)):
    for j in range(len(X[i])):
        if X[i][j] != 0:
            non_zero_indices.append((i,j,X[i][j])) # Has length 1.000.209.

The above coding cell creates the array with the indices of the elements that have a rating. The code cell below is the function that randomly shuffles the data and splits it into 5 parts.

In [11]:
def k_fold_cross_validation(data,k):
    np.random.shuffle(data)
    splitted_data = []
    for i in range(k):
        splitted_data.append(data[int(len(data)/k*i):int(len(data)/k*(i+1))])
        print('Size set {0}: {1}'.format(i+1,np.shape(splitted_data[i])))
    return splitted_data    

In [12]:
folded_data = k_fold_cross_validation(non_zero_indices,k=5)

Size set 1: (200041, 3)
Size set 2: (200042, 3)
Size set 3: (200042, 3)
Size set 4: (200042, 3)
Size set 5: (200042, 3)


Above is some output that displays the size of each of the 5 sets. Each set contains approximately $2*10^5$ datapoints/ratings.

Now we have set up our data in 5 different parts. Next is the process called matrix factorization.

### 3. Matrix factorization.

The following equations are taken from the gravity-tikk paper page 24.

The matrix $X$ can be approximated as the product of two matrices:

$$X \approx UM $$

$X$ has shape $(i,j)$ where $i$ denotes the $i$th user and $j$ denotes the $j$th movie. The element $(i,j)$ contains the rating the user has given the movie. The matrices $U$ and $M$ have shapes $(i,k)$ and $(k,j)$, respectively. $k$ is a factor which can be chosen. In the code this will simply be called $k$ and experimentation will be done with this value. Now, instead of having a single matrix with approximate size $(6000,4000)$, you have two matrices with sizes $(6000,k)$ and $(k,4000)$, which is much less data. The key idea is that the product of the matrices $U$ and $M$ will give accurate predictions of what rating a certain user will give to movie he hasn't rated yet, based on his/hers known ratings.

How do we obtain the matrix elements of $U$ and $M$? We initialize the matrices with random weight produced by numpy.random.randn, which are all values obtained from a gaussian distribution. The elements of $U$ and $M$ are then constantly updated over several iterations. They are computed with the following equations.

$$ \hat{x}_{ij} = \sum_{k=1}^{K} = u_{ik}m_{kj} $$

$ \hat{x}_{ij} $ denotes the predicted values of what user $i$ will rate movie $j$. Next the error of the predicted value compared to the known value is computed. This is done only for values which are known, so for the total $1000209$ ratings given by the $6041$ users.

$$ e_{ij} = x_{ij}-\hat{x}_{ij} $$

The total error of all these ratings is just the sum of the errors:

$$ SE = \sum_{ij} e^2_{ij} $$

What you want, is to minimize the total error over time. The new values for the elements in the matrices $U$ and $M$ are computed with the two following algorithms:

$$ u`_{ik}  = u_{ik} + \eta \cdot (2 e_{ij} \cdot m_{kj} - \lambda \cdot u_{ik} ) $$
$$ m`_{kj}  = m_{kj} + \eta \cdot (2 e_{ij} \cdot u_{ik} - \lambda \cdot m_{kj} ) $$

$ u`_{ik} $ is the updated parameter value. $\eta$ is the learning rate, so it specifies how fast the algorithm learns. $\lambda$ is the regularization factor to prevent large weights.

After these computations are performed on the training set, the new matrices $U$ and $M$ are computed and the equations that compute the predicted matrix values $\hat{x}_{ij}$ and the (total) error is computed on the test set. From this error the RMSE can be computed. This is just the square root of the mean of the total error. This value usually starts out high, around 3 are 4, then rapidly decreases in the first few iterations and then slowly drops to 0.9 or 0.8, depending on the initial parameter values for the learning rate and regularization value. The iteration process will be stopped of the error increases. That means no further improvement can be obtained using the algorithm.

In [29]:
def mat_fac(indices,X,test_set):
    # Initial parameters. Set eta and lambda to some small positive values.
    eta = 0.01 # Learning rate (eq. 6 gravity-tikk).
    lamda = 0.1 # The regularization factor lambda (eq. 6 gravity-tikk).
    k = 10 # The parameter that sets the shape of the matrices U and M.
    n_iter = 100 # Number of maximum iterations.

    # Initialize the two matrices (eq. 1 gravity-tikk)
    U = np.random.randn(users['UserID'][len(users)-1]+1,k) # Initialize the weights of U ...
    M = np.random.randn(k,movies['MovieID'][len(movies)-1]+1) # ... and M randomly.
    
    prev_RMSE = 10e10+1; RMSE = 10e10 # Set some initial values for the RMSE.
    prev_RMSE_test = 10e10+1; RMSE_test = 10e10
    
    total_SE_test = []
    
    for n in range(n_iter):
        if RMSE_test < prev_RMSE_test: # Only continue the loop if there is improvement.
            prev_RMSE_test = RMSE_test
            SE_test = 0; SE = 0 # The total squared error (which is equivalent to minimize RMSE).
            for i in range(len(indices)): # Loop until the terminal condition is met.
                if i != test_set: # Don't use the testing set for training.
                    for j in range(len(indices[i])):
                        x_hat_ij = np.dot(U[indices[i][j][0],:],M[:,indices[i][j][1]]) # Eq. 3 gravity-tikk
                        e_ij = X[indices[i][j][0]][indices[i][j][1]]-x_hat_ij # Eq. 4 gravity-tikk
                        SE += e_ij**2; # Total error.
                        temp = U[indices[i][j][0],:]+eta*(2*e_ij*M[:,indices[i][j][1]]-lamda*U[indices[i][j][0],:])
                        M[:,indices[i][j][1]] += eta*(2*e_ij*U[indices[i][j][0],:]-lamda*M[:,indices[i][j][1]])
                        U[indices[i][j][0],:] = temp # Update factorized matrices.
            X_pred = np.dot(U,M)
            for j in range(len(indices[test_set])):
                SE_test += (X[indices[test_set][j][0]][indices[test_set][j][1]]-X_pred[indices[test_set][j][0]][indices[test_set][j][1]])**2  
                RMSE_test = (SE_test/len(indices[test_set]))**0.5
            RMSE = np.sqrt(SE/800168) # Compute the root mean squared error. (Use a fixed number for efficiency.)
            total_SE_test.append(SE_test) # For statistics.
            sys.stdout.write('\rIterations: {0}\n'.format(n)) # Overview of the process.
            sys.stdout.write('\rSE_train: {0}. RMSE_train: {1}\n'.format(SE,RMSE)) # Overview of the process.
            sys.stdout.write('\rSE_test:  {0}. RMSE_test:  {1}\n\n'.format(SE_test,RMSE_test)) # Overview of the process.        
        else:
            print('\nError increased. Process terminated.\n')
            break;
            
    return U,M,total_SE_test

### Results:

In [28]:
U_0,M_0,SE_0 = mat_fac(folded_data,X,0) # Done with 0.001,0.01,10,100. Gives 0.928598

Iterations: 0
SE_train: 14064515.323242577. RMSE_train: 4.19248768560201
SE_test:  2899222.06207733. RMSE_test:  3.806985581381573

Iterations: 1
SE_train: 7620082.536808458. RMSE_train: 3.085952578915223
SE_test:  1107361.1546118236. RMSE_test:  2.3528006631485407

Iterations: 2
SE_train: 2912483.3763803593. RMSE_train: 1.9078364327441928
SE_test:  562710.1741828651. RMSE_test:  1.677192359629935

Iterations: 3
SE_train: 1693423.4522644007. RMSE_train: 1.4547628277505065
SE_test:  393731.3231178715. RMSE_test:  1.402944447830704

Iterations: 4
SE_train: 1253417.0734061184. RMSE_train: 1.2515759620798044
SE_test:  318899.61216077214. RMSE_test:  1.2626049483890212

Iterations: 5
SE_train: 1043114.7802550715. RMSE_train: 1.14176167179436
SE_test:  278771.76077781897. RMSE_test:  1.1804969806819532

Iterations: 6
SE_train: 925903.1308480541. RMSE_train: 1.075702521619204
SE_test:  254630.56191044566. RMSE_test:  1.128225095767485

Iterations: 7
SE_train: 853834.196942427. RMSE_train: 1.0

Iterations: 61
SE_train: 592739.0884203017. RMSE_train: 0.86067897568289
SE_test:  177508.2576400182. RMSE_test:  0.9419975475165994

Iterations: 62
SE_train: 591678.8587589926. RMSE_train: 0.859908884916167
SE_test:  177322.10083544877. RMSE_test:  0.9415034711970303

Iterations: 63
SE_train: 590635.5249540439. RMSE_train: 0.8591503924920257
SE_test:  177140.15464233307. RMSE_test:  0.9410203195293411

Iterations: 64
SE_train: 589608.5854315872. RMSE_train: 0.8584031640384123
SE_test:  176962.28172442308. RMSE_test:  0.9405477443808613

Iterations: 65
SE_train: 588597.5539537853. RMSE_train: 0.8576668747386403
SE_test:  176788.34791787594. RMSE_test:  0.940085404988718

Iterations: 66
SE_train: 587601.9600471273. RMSE_train: 0.856941209727455
SE_test:  176618.2224075191. RMSE_test:  0.939632968484996

Iterations: 67
SE_train: 586621.3492397607. RMSE_train: 0.8562258643466908
SE_test:  176451.7778730104. RMSE_test:  0.9391901103408525

Iterations: 68
SE_train: 585655.2831361599. RMSE_t

In [30]:
U_0,M_0,SE_0 = mat_fac(folded_data,X,0) # Done with 0.01,0.1,10,100. Gives 0.884577

Iterations: 0
SE_train: 3563131.281638583. RMSE_train: 2.110208277981861
SE_test:  218363.4999660699. RMSE_test:  1.044793627046562

Iterations: 1
SE_train: 758722.2108931269. RMSE_train: 0.9737574856461078
SE_test:  194973.27318762627. RMSE_test:  0.9872520242032813

Iterations: 2
SE_train: 712640.6723028921. RMSE_train: 0.9437233765665682
SE_test:  188592.10549669852. RMSE_test:  0.9709620281942815

Iterations: 3
SE_train: 693177.3835320486. RMSE_train: 0.930746908968955
SE_test:  184516.1861338203. RMSE_test:  0.9604123282954209

Iterations: 4
SE_train: 677328.8656294869. RMSE_train: 0.9200452817875571
SE_test:  180999.01673359226. RMSE_test:  0.9512148010309935

Iterations: 5
SE_train: 662990.4625445812. RMSE_train: 0.9102549531444387
SE_test:  178020.4685762454. RMSE_test:  0.9433556642644494

Iterations: 6
SE_train: 650482.453990434. RMSE_train: 0.9016276125397816
SE_test:  175542.38310307692. RMSE_test:  0.9367667913899314

Iterations: 7
SE_train: 639624.8201966492. RMSE_train: 

Iterations: 61
SE_train: 536045.3853405264. RMSE_train: 0.8184839945321495
SE_test:  156902.5091938025. RMSE_test:  0.8856363553171648

Iterations: 62
SE_train: 535836.4807025524. RMSE_train: 0.8183244914664294
SE_test:  156880.9162166919. RMSE_test:  0.8855754124309141

Iterations: 63
SE_train: 535636.9641080613. RMSE_train: 0.8181721273379988
SE_test:  156860.56425032482. RMSE_test:  0.8855179682687631

Iterations: 64
SE_train: 535446.3014791866. RMSE_train: 0.8180264981680759
SE_test:  156841.36503705816. RMSE_test:  0.8854637743779612

Iterations: 65
SE_train: 535263.995141225. RMSE_train: 0.8178872273290734
SE_test:  156823.23706817388. RMSE_test:  0.8854126012592114

Iterations: 66
SE_train: 535089.5811328542. RMSE_train: 0.8177539635438281
SE_test:  156806.10508948102. RMSE_test:  0.8853642369848796

Iterations: 67
SE_train: 534922.6266785109. RMSE_train: 0.8176263790020808
SE_test:  156789.89962992843. RMSE_test:  0.8853184858802208

Iterations: 68
SE_train: 534762.7278247107. 

In [13]:
plt.figure(figsize=(12,6))
plt.plot(np.arange(0,len(SE_0)),SE_0,'.')
plt.xlabel('Iterations'); plt.ylabel('Total error')
plt.title('Total error over time')
plt.show(); plt.close()
################ might be a good idea to plot the RMSE rather than total SE

NameError: name 'SE_0' is not defined

<Figure size 864x432 with 0 Axes>

Above is an example plot which shows the error over time. Notice the rapid decrease in the beginning and the slow decrease of the error in the final iterations.

We ran the algorithm with different inial settings. See the two tables below for the results:

One-time run with different initial parameter settings.

| Learning rate $\eta$ | Regularization factor $\lambda$ | Factor $k$ | Iterations | RMSE |
| --- | --- | --- | --- | --- |
| 0.001 | 0.01 | 10 | 100 | 0.9286 |
| 0.01 | 0.1 | 10 | 100 | 0.8846 |
| 0.03 | 0.3 | 10 | 100 | 0.9926 |

5-fold cross validation run with the same initial settings.

| Learning rate $\eta$ | Regularization factor $\lambda$ | Factor $k$ | Iterations | RMSE |
| --- | --- | --- | --- | --- |
| 0.005 | 0.05 | 12 | 100 | 0.8884 |
| 0.005 | 0.05 | 12 | 100 | 0.8808 |
| 0.005 | 0.05 | 12 | 100 | 0.8848 |
| 0.005 | 0.05 | 12 | 100 | 0.8846 |
| 0.005 | 0.05 | 12 | 100 | 0.8853 |

### Discussion:

- We played a bit around with the $\eta$ and $\lambda$ values as you can see in the first table. The slides instructed $\eta = 0.001$ and $\lambda = 0.01$ as example parameter values. However, if we make both these values 10 times as large, we obtain a lower error after 100 iterations. This could mean these values are just better for our case, or that with lower parameter values, more iterations are needed to obtain the same accurate results.

- One odd thing that occurs, is that the root mean squared error on the test set always decreases. This is peculiar, because if you keep training your data on the training set and test it simultaneously in the test set, you would expect there to be a point in time where an increase in accuracy on the training set won't give improvement on the test set. Unless we give the learning rate and regularization factor some absurd high values, the loop never terminates because of increasement in the error. We don't know exactly what causes this behaviour. However, if you take a look at the graph, the RMSE over time behaves as expected. To be more clear, the error starts at a relatively high value, which makes sense because the matrices $U$ and $M$ are randomly initialized from a gaussian distribution. The algorithm then quickly decreases the error, and in the end the error does not change much anymore. This curve is what you would expect, and since the final RMSE we obtained after 100 iterations looks really well, we'd argue the odd behaviour is not a large concern.

- In the second table, where we tested the 5-fold cross validation, you can see that the RMSE for all the 5 cases are really close to each other. This makes sense, since each test set contains approximately $2 \cdot 10^5$ elements and they are all randomly distributed. We also changed the factor $k$ to a larger number, to see the effect of this. A larger $k$ factor means $\hat{x}_{ij}$ can be approximated more accurate, but it also means computations take longer. This is a trade-of which needs to be considered.

- We think it is possible to obtain a lower RMSE using this method. For example, the number of iterations can be larger, the factor $k$ can be larger, and the parameter values $\eta$ and $\lambda$ can be tweaked more carefully. However, this all takes a lot of time, and since a single run already takes an hour, we haven't optimized the parameter values 100 percent. Also, the result $ \text{RMSE} = 0.89 $ on the test set is already a large improvement compared to the naive models, so we were happy with that result.

- The naive approaches were able to get fairly low RMSE values even though they were extremely simple methods of prediction. Due to their simplicity, their computation times were extremely short, indicating that these approaches are good starting points for handling data such as these. However, when compared to the Matrix Factorization approach these naive approaches score much worse. The Matrix Factorization, though much more computationally expensive, was able to acquire an average of Root Mean Square errors of $~0.82 $ on the training sets and $~0.89 $ on the test sets, this is a $~10\%$ improvement of the best naive method (the Regression Model) and a staggering $~29\%$ improvement of the global mean approach.


### Conclusion:

In conclusion, the naive methods, though true to their names, are fast and deliver relatively good RMSE values. However, when trading simplicity and short computation times for complexity and long computation times respectively, we were able to make a large improvement in the RMSE using the Matrix Factorization technique with a gradient descent algorithm.